In [9]:
#!/usr/bin/env python

import math
from scipy.stats import norm
import numpy as np
import sys
sys.path.insert(1, '../../../Programas_Cosmologia/PySSC/')
import PySSC
import ncounts_gauss_SSC
try:
    import gi

    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())


# Initialize the `Cosmological model`

In [2]:
cosmo = Nc.HICosmoDEXcdm()
cosmo.props.H0  =  67.81
cosmo.props.Omegac  =  0.2612
cosmo.props.Omegab =  0.0486
cosmo.props.Omegax = 1 - 0.0486 -  0.2612
cosmo.props.w = -1.0


reion = Nc.HIReionCamb.new () 
prim = Nc.HIPrimPowerLaw.new () 



cosmo.add_submodel (reion)
cosmo.add_submodel (prim)

dist = Nc.Distance.new (2.0)

tf = Nc.TransferFunc.new_from_name ("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new (tf)
psml.require_kmin (1.0e-6)
psml.require_kmax (1.0e3)

psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()
prim.props.n_SA = 0.9660
print(0.82505858,cosmo.sigma8(psf),cosmo.Omega_k0())

old_amplitude = math.exp (prim.props.ln10e10ASA)
prim.props.ln10e10ASA = math.log ((0.82505858/ cosmo.sigma8(psf))**2 * old_amplitude)
print(0.82505858, cosmo.sigma8(psf))

0.82505858 0.8900071895091025 -9.085228043836935e-05
0.82505858 0.82505858


# Initialize the `ClusterAbundance` object

In [3]:
mulf = Nc.MultiplicityFuncTinker.new ()
mulf.set_mdef (Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta (200.0)

hmf = Nc.HaloMassFunction.new (dist, psf, mulf)
hbias_Tinker = Nc.HaloBiasTinker.new(hmf)
ca = Nc.ClusterAbundance.new(hmf,hbias_Tinker)
ca.set_area(0.1)

# Define proxy modelling


In [4]:
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (math.log(10)*np.log10(1e14),math.log(10)*np.log10(1e16)))
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (0.25,2))

# Construct the $S_{ij}$  Matrix

# Data model

$f(x_i)$ for the Gaussian distribution $$-2\ln L = \sum_{ij}\left[(f(x_i) - y_i)C^{-1}_{ij}(f(x_j) - y_j)\right] + \dots$$

In [10]:
z_obs_bins  = Ncm.Vector.new_array(np.linspace(0.1,0.8,8))
lnM_obs_bins = Ncm.Vector.new_array(np.linspace(14.8,15, 2))
mset = Ncm.MSet.new_array([cosmo,cluster_m,cluster_z])


gauss = ncounts_gauss_SSC.ncounts(len = (z_obs_bins.len()-1)* (lnM_obs_bins.len()-1))
gauss.set_z_obs_bins(z_obs_bins)
gauss.set_lnM_obs_bins(lnM_obs_bins)
gauss.set_cad(ca)
gauss.set_S(Ncm.Matrix.new0((z_obs_bins.len()-1)* (lnM_obs_bins.len()-1) , (z_obs_bins.len()-1)* (lnM_obs_bins.len()-1)))


rng = Ncm.RNG.seeded_new (None, 123)
gauss.resample (mset, rng)

dset = Ncm.Dataset.new()
dset.append_data(gauss)

# Creating the fit object

In [7]:

cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
cosmo.props.w_fit = True
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8)

print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()
#gauss.do_cov_func(mset)

lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)


gauss.y.log_vals("y: ", "%.3f", True)
gauss.cov.log_vals("cov:", "%.3f")


0.82505858
y: 70223.440 176517.514 312984.063 464800.255 621161.886 766849.292 898732.675
cov: 70511.766 0.000 0.000 0.000 0.000 0.000 0.000
cov: 0.000 176279.838 0.000 0.000 0.000 0.000 0.000
cov: 0.000 0.000 313392.592 0.000 0.000 0.000 0.000
cov: 0.000 0.000 0.000 465987.860 0.000 0.000 0.000
cov: 0.000 0.000 0.000 0.000 621104.084 0.000 0.000
cov: 0.000 0.000 0.000 0.000 0.000 768095.243 0.000
cov: 0.000 0.000 0.000 0.000 0.000 0.000 898681.031


# Fisher Matrix

In [8]:
fit.log_info()
fit.fisher()
fit.log_covar ()

#----------------------------------------------------------------------------------
# Data used:
#   - ncounts_gauss_SSC+ncounts
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.81               [FIXED]
#   -     Omegac[01]:  0.2612              [FREE]
#   -     Omegax[02]:  0.6902              [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0486              [FIXED]
#   -          w[07]: -1                   [FREE]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#-------------------------------------------------------------------